In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, kruskal, f_oneway
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.ensemble import RandomForestRegressor
import sys,os

In [2]:
'''
Data location: 'https://www.fhfa.gov/DataTools/Downloads/Pages/Public-Use-Databases.aspx'
'''
data_path = '../data/'

In [3]:
fm = pd.read_pickle('../data/fm_2020_all.gz')

## Manually label columns

In [4]:
idx_cols = {
    1: 'e_flag',
    2: 'id',
}

In [5]:
geo_cols = {
    3 : 'state_code',
    4 : 'msa_code',
    5 : 'cn_code', # county
    6 : 'ct_code', # cesnsus tract 
    58: 'rural_ct', # bool
    59: 'lower_miss_delta_ct', # bool
    60: 'middle_app_cn', # bool
    61: 'persistent_poverty_cn', #bool
    62: 'conc_pov_area', # bool
    63: 'high_opp_area', # bool
    64: 'qoz_ct', # bool
}

In [6]:
race_cols = {
    7 : 'ct_pct_minority', # float
    19: 'b_race_1st_val',
    20: 'b_race_2nd_val',
    21: 'b_race_3rd_val',
    22: 'b_race_4th_val',
    23: 'b_race_5th_val',
    24: 'b_ethnicity',
    25: 'cb_race_1st_val',
    26: 'cb_race_2nd_val',
    27: 'cb_race_3rd_val',
    28: 'cb_race_4th_val',
    29: 'cb_race_5th_val',
    30: 'cb_ethnicity',
}

In [7]:
age_gender_cols = {
    31: 'b_gender',
    32: 'cb_gender',
    33: 'b_age_gr', # ord
    34: 'cb_age_gr', 
    40: 'b_gt_62', # bool
    41: 'cb_gt_62',
    42: 'ltv_at_orig' # float
} 

In [8]:
income_cols = {
    8 : 'ct_med_income', # int
    9 : 'msa_med_income', # int
    10: 'ct_med_inc/msa_med_inc', # X
    11: 'b_income', # int
    12: 'msa_med_income_2', # X
    13: 'b_income_ratio', # float
    14: 'acquired_unpaid_balance_gr', # int
}

In [9]:
loan_cols = {
    15: 'loan_purpose', # cat
    16: 'fed_guar', # cat
    17: 'n_borrowers', # ord
    18: 'ft_home_buyer', # bool,
    35: 'occ_code', # cat
    36: 'rate_spread', # num
    37: 'hoepa_status', # cat
    38: 'property_type', # cat 
    39: 'lien_status', # X
    43: 'mortgage_note_origin', # cat
    44: 'mortgage_term_mo', # num
    45: 'n_units', # ord
    46: 'ir_orig', # label column, binarize
    47: 'note_amt_gr', # num
    48: 'preapproval', # cat
    49: 'app_chan', # cat
    50: 'aus_name', # cat
    51: 'b_cs_model', # cat
    52: 'cb_cs_model', # X
    53: 'debt_to_income_ratio_gr', # ord
    54: 'discount_pts', # num
    55: 'intro_rate_period', # X
    56: 'land_ownership', # cat
    57: 'property_value_gr', # num
}

In [10]:
all_cols = {**idx_cols, **geo_cols, **age_gender_cols, **race_cols, **income_cols, **loan_cols}

In [11]:
ordinal_cols = [
    'b_age_gr',
    'n_borrowers',
    'n_units',
    'debt_to_income_ratio_gr',
]

In [12]:
cat_cols = [
    'loan_purpose',
    'fed_guar',
    'occ_code',
    'hoepa_status',
    'property_type',
    'mortgage_note_origin',
    'preapproval',
    'aus_name',
    'b_cs_model',
    'land_ownership',
    'app_chan'
]

In [13]:
cont_cols = [
    'ct_pct_minority',
    'ct_med_income',
    'msa_med_income',
    'b_income',
    'b_income_ratio',
    'acquired_unpaid_balance_gr',
    'rate_spread',
    'mortgage_term_mo',
    'note_amt_gr', 
    'discount_pts',
    'property_value_gr',
    'ltv_at_orig'
]

### Race

In [15]:
race_cols

{7: 'ct_pct_minority',
 19: 'b_race_1st_val',
 20: 'b_race_2nd_val',
 21: 'b_race_3rd_val',
 22: 'b_race_4th_val',
 23: 'b_race_5th_val',
 24: 'b_ethnicity',
 25: 'cb_race_1st_val',
 26: 'cb_race_2nd_val',
 27: 'cb_race_3rd_val',
 28: 'cb_race_4th_val',
 29: 'cb_race_5th_val',
 30: 'cb_ethnicity'}

In [16]:
np.unique(fm.loc[:,19],return_counts=True)

(array([1, 2, 3, 4, 5, 6, 7, 9], dtype=int32),
 array([  12345,  377371,  155100,   10369, 3558752,  657081,       9,
          86597]))

In [196]:
print(
    19, np.unique(fm.loc[:,19]),race_cols[19], '\n',
    20, np.unique(fm.loc[:,20]),race_cols[20], '\n',
    21, np.unique(fm.loc[:,21]),race_cols[21], '\n',
    22, np.unique(fm.loc[:,22]),race_cols[22], '\n',
    23, np.unique(fm.loc[:,23]),race_cols[23], '\n',
)

19 [1 2 3 4 5 6 7 9] b_race_1st_val 
 20 [1 2 3 4 9] b_race_2nd_val 
 21 [1 2 3 9] b_race_3rd_val 
 22 [1 2 9] b_race_4th_val 
 23 [1 9] b_race_5th_val 



In [197]:
pct_id_white = len(fm.loc[fm[19]==5,1])/len(fm)
pct_id_white

0.732611663644613

In [198]:
np.unique(fm.loc[fm[19]==5,24],return_counts=True)

(array([1, 2, 3, 9], dtype=int32), array([ 350041, 3107973,   86842,   13896]))

In [199]:
# whte + hispanic
len(fm.loc[(fm[19]==5) & (fm[24]==1),1])/len(fm)

0.07206012651452644

In [200]:
# white + non-hispanic
len(fm.loc[(fm[19]==5) & (fm[24]==2),1])/len(fm)

0.6398134149534834

In [202]:
sum(r_white_non_hisp)/len(r_white_non_hisp)

0.6398134149534834

In [203]:
pct_id_black = len(fm.loc[(fm[19]==3) | (fm[20]==3) | (fm[21]==3),1])/len(fm)
pct_id_black

0.03326399902503775

In [205]:
sum(r_black)/len(r_black)

0.03326399902503775

In [206]:
# black + hispanic
len(fm.loc[(fm[19]==3) & (fm[24]==1),1])/len(fm)

0.0012711976060724337

In [207]:
# black + non-hispanic
len(fm.loc[(fm[19]==3) & (fm[24]==2),1])/len(fm)

0.02930053869957823

In [208]:
pct_id_asian = len(fm.loc[(fm[19]==2) | (fm[20]==2) | (fm[21]==2) | (fm[22]==2),1])/len(fm)
pct_id_asian

0.08303153969924391

In [209]:
pct_id_white + pct_id_black + pct_id_asian

0.8489072023688947

In [210]:
# no data + n/a
len(fm.loc[fm[19]==6,1])/len(fm)

0.1352679828656973

In [211]:
# n/a + unknown
len(fm.loc[(fm[19]==9) | (fm[19]==7) | (fm[19]==6),1])/len(fm)

0.153096863816549

Create columns

In [201]:
r_white_non_hisp = ((fm[19]==5) & (fm[24]==2)).astype(int)

In [204]:
r_black = ((fm[19]==3) | (fm[20]==3) | (fm[21]==3)).astype(int)

### Gender

In [214]:
np.unique(fm.loc[:,31],return_counts=True)

(array([1, 2, 3, 4, 9], dtype=int32),
 array([2992720, 1464262,  370312,       9,   30321]))

In [228]:
gender_m = (fm[31]==1).astype(int)

In [229]:
gender_f = (fm[31]==2).astype(int)

### Apply

In [450]:
new_cols = set(fm.columns) - set(list(range(1,7)) + list(range(19,31)) + [55,12,31,39,40,10])
verbose_cols = [all_cols[c] for c in new_cols]
fm_new = fm.loc[:,new_cols]
fm_new.columns = verbose_cols
## remove co-borrower info
fm_new = fm_new.loc[:,[v for v in verbose_cols if 'cb' not in v]]

In [451]:
fm_new['r_white_non_hisp'] = r_white_non_hisp
fm_new['r_black'] = r_black
fm_new['gender_m'] = gender_m
fm_new['gender_f'] = gender_f

In [452]:
fm_new_cut = fm_new.loc[((fm_new['r_white_non_hisp'] ==1) | (fm_new['r_black'] ==1))
           & ((fm_new['gender_m']==1)| (fm_new['gender_f']==1)),:]

In [453]:
len(fm_new_cut)/len(fm_new)

0.6689997002649856

In [496]:
set(fm_new.columns) -  set(ordinal_cols + cat_cols + cont_cols)

{'conc_pov_area',
 'ft_home_buyer',
 'gender_f',
 'gender_m',
 'high_opp_area',
 'ir_orig',
 'lower_miss_delta_ct',
 'middle_app_cn',
 'persistent_poverty_cn',
 'qoz_ct',
 'r_black',
 'r_white_non_hisp',
 'rural_ct'}

In [419]:
len(fm_new.columns)

40

In [420]:
# fm_new_filter['qoz_ct'].hist()

In [422]:
# np.unique(fm_new_filter['b_gt_62'],return_counts=True)

In [455]:
# TODO: add ordinal, categorical and normalization
from sklearn.preprocessing import LabelBinarizer, StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline

In [456]:
oh = OneHotEncoder(handle_unknown='ignore')
oc = OrdinalEncoder(handle_unknown='ignore')
mm = MinMaxScaler()

In [457]:
ct = ColumnTransformer(
    [('oh',oh, cat_cols),
    ('oc',oc,  ordinal_cols),
    ('mm',mm, cont_cols)],remainder='passthrough')

In [503]:
X = fm_new_cut.drop('ir_orig',axis=1)

In [505]:
ct_out = ct.fit_transform(X)

In [506]:
ct_out.shape

(3249749, 62)

In [520]:
len(oh.fit(X[cat_cols]).get_feature_names(cat_cols)) + len(set(X) - set(cat_cols) )

62

In [534]:
ct_cols = list(oh.fit(X[cat_cols]).get_feature_names(cat_cols))\
    + ordinal_cols + cont_cols + \
    list(X.drop(cat_cols + ordinal_cols + cont_cols,axis=1).columns) 

In [538]:
list(X.drop(cat_cols + ordinal_cols + cont_cols,axis=1).columns) 

['ft_home_buyer',
 'rural_ct',
 'lower_miss_delta_ct',
 'middle_app_cn',
 'persistent_poverty_cn',
 'conc_pov_area',
 'high_opp_area',
 'qoz_ct',
 'r_white_non_hisp',
 'r_black',
 'gender_m',
 'gender_f']

In [535]:
len(ct_cols)

62

In [536]:
ct_f = pd.DataFrame(data=ct_out,columns=ct_cols)

In [545]:
ct_f['ft_home_buyer'] = ct_f['ft_home_buyer'].apply(lambda r: 1 if r==1 else 0)

### Interest rate division threshold
 - divide equally based on interest rate

In [546]:
fm_new_cut.loc[:,'ir_orig'].median(), fm_new_cut.loc[:,'ir_orig'].mean()

(3.12, 3.210376302883245)

In [549]:
ct_y = pd.cut(fm_new_cut.loc[:,'ir_orig'],bins=[0,3.12,7],labels=[0,1])

### Test train split

In [580]:
from math import floor
def test_train_split(X,y,train_frac=0.8):
    train_size = floor(len(X)*train_frac)
    test_size = len(X) - train_size
    s_X = X.sample(frac=1,random_state=23)
    s_y = y.sample(frac=1,random_state=23)
    X_train = s_X[:train_size]
    y_train = s_y[:train_size]
    X_test = s_X[train_size:len(X)]
    y_test = s_y[train_size:len(y)]
    return X_train, y_train, X_test, y_test

In [581]:
X_train, y_train, X_test, y_test = test_train_split(ct_f,ct_y)

In [582]:
X_train.shape, y_train.shape

((2599799, 62), (2599799,))